# Hosting multiple models with multi-model endpoints

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r model_a_s3_path
%store -r model_b_s3_path

In [ ]:
%store -r s3_bucket
%store -r prefix

In [ ]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

In [ ]:
from sagemaker.image_uris import retrieve

image_uri = retrieve(
    "xgboost", 
    region="us-east-1", 
    version="0.90-2"
)

image_uri

In [ ]:
models_path = f"s3://{s3_bucket}/model-artifacts/"

In [ ]:
from sagemaker.multidatamodel import MultiDataModel

multi_model = MultiDataModel(
    name="chapter09-multi",
    model_data_prefix=models_path, 
    image_uri=image_uri,
    role=role
)

In [ ]:
multi_model.add_model(model_a_s3_path)
multi_model.add_model(model_b_s3_path)

In [ ]:
list(multi_model.list_models())

In [ ]:
model_a, model_b = list(
    multi_model.list_models()
)

print(model_a)
print(model_b)

In [ ]:
%%time

endpoint_name = "chapter09-mma"

multi_model.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium', 
    endpoint_name=endpoint_name
)

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = Predictor(
    endpoint_name=endpoint_name
)

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [ ]:
a, b = list(multi_model.list_models())

In [ ]:
a

In [ ]:
b

In [ ]:
predictor.predict(data="10,-5", target_model=a)

In [ ]:
predictor.predict(data="10,-5", target_model=b)

In [ ]:
predictor.delete_endpoint()